# NE 630 - Lesson 20

**Last Time**: Parametric studies and optimization of unit cells.

**Today**: reactivity feedback

## What is Reactivity?

Recall that

$$
  k_{\infty} = \frac{\text{total fission rate}}{\text{total absorption rate}} \, .
$$

Then the *reactivity* is defined as

$$
  \rho_{\infty} = \frac{k_{\infty} - 1}{k_{\infty}} \, 
  \tag{Equivalent to 9.1}
$$

**Example**: Compute the reactivity of a reactor with $k_{\infty} = 1.01$.

In [1]:
rho = (1.01-1)/1.01
rho

0.00990099009900991

Reactivity, like the multiplication factor, is *unitless*, but we often label it with the unit $[dk/k]$. There are also special "units" we use to scale reactivities numerically:

  - A *dollar* (\\$) of reactivity is numerically equal to $\beta$, the fraction of fission neutrons that are *delayed*.
  - A *pcm* (percent-milli) of reactivity is numerically equal to $10^{-5} = 10^{-2} 10^{-3} = 1~\text{m}\%$ (where m is for milli)

**Example**: How many dollars of reactivity is $\rho = 0.01$ dk/k?  pcm?

In [2]:
beta = 0.007 # for U-235
rho = 0.01 # dk/k
rho_dollars = rho / beta
rho_dollars
rho_pcm = rho / 1e-5
rho_pcm

999.9999999999999

## Reactivity Coefficients

We often care how $\rho$ changes when a system property (like temperature) changes.  In other words, we want to compute for some property $x$ the following:

$$
   \frac{d\rho}{dx} = \frac{d\rho}{dk} \frac{\partial k}{\partial x}
   \tag{***}
$$

The first term on the right is

$$
   \frac{d\rho}{dk} = \frac{1}{k^2} \approx \frac{1}{k} 
   \tag{Equivalent to 9.2} 
$$

where the $\approx$ is really only justified for nearly-critical systems!

Then, we can write \*\*\* as 

$$
   \frac{d\rho}{dx} \approx \frac{1}{k} \frac{\partial k}{\partial x} 
     = \frac{1}{\varepsilon}\frac{\partial \varepsilon}{\partial x} 
     + \frac{1}{p}\frac{\partial p}{\partial x}
     + \frac{1}{\eta_T}\frac{\partial \eta_T}{\partial x}
     + \frac{1}{f}\frac{\partial f}{\partial x}
   \tag{Equivalent to 9.10}
$$

This is a *reactivity coefficient*, and given $\delta x$, we want $\delta \rho$.

## Fuel Temperature Coefficient

The FTC is dominated by changes in resonance absorption via Doppler broadening.

A simple approximation for $p$ is 

$$
  p = \exp \left ( - \frac{V_f N_{fe}}{V_m \xi^m \Sigma^m_s} I(T_f) \right )
\tag{4.40}
$$

where 

$$
  I(T_f) \approx I(300) [ 1 + \bar{\gamma} ( \sqrt{T_f} - \sqrt{300}) \, ,
  \tag{9.13}
$$

$$
  \bar{\gamma} = 0.0061 + 0.0047 (4/\rho D) \, ,
  \tag{Table 9.1}
$$

and 

$$
  I(300) = 4.45 + 26.6\sqrt{4/\rho D} \, ,
  \tag{Table 4.3}
$$

is the resonance integral (in barns!).




With these expressions, the fuel-temperature coefficient is

$$
  \frac{d\rho}{dT_f} \equiv \alpha_{f} = 
    - \frac{\bar{\gamma}}{2\sqrt{T_f}}\ln(1/p(T_0)) \, .
   \tag{9.16}
$$ 

Remember that this coefficient is a function of $T_f$.

## Exercise

Compute $\alpha_f$ for our PWR previously analyzed with `LWRUnitCell`.  Use the default settings.

**Solution**

We can actually use the `LWRUnitCell` class to get many of the parameters needed.

In [3]:
import sys
sys.path.append("unitcell") 
from unitcell_utils import *

import warnings                    # THIS IS GENERALLY NOT RECOMMNENDED
warnings.filterwarnings("ignore")  # BUT NOBODY WILL DIE....

In [4]:
pwr = LWRUnitCell(T_fuel=900)

In [5]:
V_f = pwr.volume['fuel']
V_m = pwr.volume['non-fuel']
xi = 0.94 # from Table 3.1
D = 2*pwr.r_fuel
N_fe = pwr.fuel.get_nuclide_atom_densities()['U238']
rho_f = pwr.fuel.density
I_300 = (4.45 + 26.6*np.sqrt(4/rho_f/D))*1e-24
N_H1 = pwr.coolant.get_nuclide_atom_densities()['H1']
N_O16 = pwr.coolant.get_nuclide_atom_densities()['O16']
Sigma_sm = (47.7*N_H1 + 4.01*N_O16)*1e-24 # sigma_s values from Table 3.2

In [6]:
p_300 = np.exp(-V_f*N_fe*I_300/(V_m*xi*Sigma_sm))
p_300

0.8337009704199803

In [7]:
bar_gamma = 0.0061 + 0.0047 * 4 / (rho_f*D)

In [8]:
α_f = -bar_gamma/(2*np.sqrt(900))*np.log(1/p_300)

In [9]:
α_f

-2.482331866969493e-05

In [1]:
print("α_f = {:.3f} pcm/K".format(α_f/1e-5))

NameError: name 'α_f' is not defined

We can also use OpenMC directly to compute $\alpha_f$ using the finite-difference approximation

$$
  \alpha_f \approx \frac{\rho(T_f+\Delta)-\rho(T_f)}{\Delta} = \frac{ \frac{k(T_f+\Delta)-1}{k(T_f+\Delta)} -  \frac{k(T_f)-1}{k(T_f)}}{ \Delta} = \frac{1}{\Delta} \left( \frac{1}{ k(T_f)} - \frac{1}{k(T_f+\Delta)} \right ) \, ,
$$

where $T_f = 900$ K and $\Delta$ is a suitable step size, e.g., 100 K.  However, we need to use `pwr.run(ng=N)` where `N` is large enough so that uncertainties are negligible.  I assume that `N = 1e6` is large enough (but you can also perform a sanity check by going from `1e6` to `2e6` and seeing when a further increase in `N` does not appear to change the result.